In [1]:
import time
import requests
import json
import pandas as pd

Download bulk data from binance (if needed)

In [ ]:
# Use free 'ProtonVPN' if IP not outside USA (required for binance API). 
# ****DO NOT RUN UNLESS YOU WANT TO DOWNLOAD NEW DATA****

interval = '5m'
symbol = 'ETHUSDT'
limit = 1000
# number of: (months * days * hours * minutes * seconds) * miliseconds. Right now this is 30 months of data.
start_time = int((time.time() - 30 * 30 * 24 * 60 * 60) * 1000)
candle_data = []

while True:
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}&startTime={start_time}'
    request = requests.get(url)
    data = request.json()

    if len(data) == 0:  
        break

    candle_data += data
    start_time = data[-1][0] + 1
    time.sleep(0.1)

with open(f'{symbol}.json', 'w') as f:
    json.dump(candle_data, f)

df = pd.DataFrame(candle_data, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base', 'Taker buy quote', 'Ignore'])
df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
df.to_csv(f'{symbol}.csv', index=False)